In [53]:
%pylab inline
import os
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import loompy
from riboraptor.helpers import path_leaf, mkdir_p
import glob
from linecache import getline
from ast import literal_eval

Populating the interactive namespace from numpy and matplotlib


/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['sample']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [19]:
def find_translating_ORFs(root_dir='/staging/as/skchoudh/re-ribo-analysis/hg38', ribocop_dir_name='ribocop_results_Feb2019_longest'):
    #root_dir ='/staging/as/skchoudh/re-ribo-analysis/hg38'
    orf_tsv_list = glob.glob('{}/*/{}/*_translating_ORFs.tsv'.format(root_dir, ribocop_dir_name))
    srx = [path_leaf(filepath).replace('_translating_ORFs.tsv', '') for filepath in orf_tsv_list]
    srp = [filepath.split(ribocop_dir_name)[0].split('/')[-2] for filepath in orf_tsv_list]
    #df = []
    #for filepath in tqdm(orf_tsv_list):
    #    df += [pd.read_table(filepath, usecols=['ORF_ID', 'profile']).set_index('ORF_ID')]
    return list(zip(srp, srx, orf_tsv_list))
    

In [20]:
sample_list = find_translating_ORFs()

In [58]:
annotation_filepath = '/home/cmb-06/as/skchoudh/genomes/hg38/ribocop_hg38_feb2019_annotation_longest_candidate_orfs.tsv'

In [21]:
annotation = pd.read_table('/home/cmb-06/as/skchoudh/genomes/hg38/ribocop_hg38_feb2019_annotation_longest_candidate_orfs.tsv')
annotation.transcript_id.value_counts()

ENST00000626826.1     2787
ENST00000597346.1     1225
ENST00000623130.1     1181
ENST00000623959.1      746
ENST00000623111.1      734
ENST00000624945.1      684
ENST00000604411.1      524
ENST00000458178.2      500
ENST00000262160.10     451
ENST00000330753.5      405
ENST00000369851.5      362
ENST00000609686.3      351
ENST00000558190.5      343
ENST00000614987.4      343
ENST00000357258.8      337
ENST00000501122.2      330
ENST00000377861.4      319
ENST00000623075.1      314
ENST00000552951.6      313
ENST00000295851.9      296
ENST00000355086.7      295
ENST00000611864.4      292
ENST00000623543.1      285
ENST00000624072.1      281
ENST00000379887.8      279
ENST00000429829.5      278
ENST00000394480.6      276
ENST00000315930.10     272
ENST00000623726.1      272
ENST00000607772.5      270
                      ... 
ENST00000523348.1        1
ENST00000636436.1        1
ENST00000589642.5        1
ENST00000592026.1        1
ENST00000555614.5        1
ENST00000415943.6        1
E

In [106]:
len(annotation.transcript_id.unique())

192961

In [22]:
annotation.head()

,ORF_ID,ORF_type,transcript_id,transcript_type,gene_id,gene_name,gene_type,chrom,strand,start_codon,coordinate
0,ENST00000335137.3_69091_70005_915,annotated,ENST00000335137.3,protein_coding,ENSG00000186092.4,OR4F5,protein_coding,chr1,+,ATG,69091-70005
1,ENST00000624431.1_182709_184155_399,annotated,ENST00000624431.1,protein_coding,ENSG00000279928.1,FO538757.2,protein_coding,chr1,+,ATG,"182709-182746,183114-183240,183922-184155"
2,ENST00000623834.3_185220_195411_1077,annotated,ENST00000623834.3,protein_coding,ENSG00000279457.3,FO538757.1,protein_coding,chr1,-,ATG,"185220-185350,185491-185559,187376-187577,1877..."
3,ENST00000623083.3_185220_195411_1392,annotated,ENST00000623083.3,protein_coding,ENSG00000279457.3,FO538757.1,protein_coding,chr1,-,ATG,"185220-185350,185491-185559,186317-186469,1871..."
4,ENST00000624735.1_184930_200086_1365,annotated,ENST00000624735.1,protein_coding,ENSG00000279457.3,FO538757.1,protein_coding,chr1,-,ATG,"184930-184971,184977-185049,185529-185559,1863..."


In [18]:
def create_index_for_annotation_row(row):
    coordinates = row['coordinate'].split(',')
    index = []
    for coordinate in coordinates:
        coordinate_start, coordinate_end = [int(x) for x in coordinate.split('-')]
        index+= list(range(coordinate_start, coordinate_end+1))
    return index

# Strategy

We will create different [loom](http://linnarssonlab.org/loompy/index.html) file for each ORF ID.
The problem however is, we have 2397475 ~ 2.5M ORF ids, so we'll have to have so may files.
However, we'll try to minimize the organization burden by having separating everything by transcript ID.
Still need 2.5M individual transcript ID files. 

## Potential issues.

1. If the user comes to us with their own query, there's no way for us to not force them to use
RiboCop, or at least somehow get this through querying the bam. But what easy way is out there anyway?

2. This ORF list is conservative. Ideally we have more than 10M ORFs. But probably not feasible to
support them all.

So we should make peace with these two limitations.



In [17]:
! rm outfile.loom

rm: cannot remove ‘outfile.loom’: No such file or directory


In [ ]:
transcript_id = 'ENST00000626826.1'
orfs_for_txid = annotation[annotation.transcript_id == transcript_id]

"""
# Stra
#for orf in orfs
loompy.create(filename, matrix, row_attrs, col_attrs)
with loompy.new("outfile.loom") as dsout:
    for sample in samples:
        with loompy.connect(sample) as dsin:
            logging.info(f"Appending {sample}.")
            dsout.add_columns(ds.layers, col_attrs=dsin.col_attrs, row_attrs=dsin.row_attrs)
"""
out_root_dir = '/staging/as/skchoudh/ribopod_hg38_looms/'
for line_number, row in  tqdm(orfs_for_txid.iterrows(), total=orfs_for_txid.shape[0]):
    annotation_orf_id = row['ORF_ID']
    out_dir = os.path.join(out_root_dir, annotation_orf_id)
    mkdir_p(out_dir)
    loom_file_path = os.path.join(out_dir, 'loomfile.loom')
    orf_index = create_index_for_annotation_row(row)
    chrom = row['chrom']
    sample = sample_list[0]      
    filepath = sample[2]
    line = getline(filepath, line_number+2)
    line_splitted = line.split('\t')
    file_orf_id = line_splitted[0]
    assert file_orf_id == annotation_orf_id
    profile = eval(line_splitted[-1])

    row_attrs = ['{}_{}'.format(chrom, pos) for pos in orf_index]

    row_attrs = { "chr_pos": np.array(row_attrs) }
    col_attrs = { "sample":  np.array([sample[0]])}
    profile = np.array([profile]).reshape(len(row_attrs['chr_pos']), 1)
    loompy.create(loom_file_path, profile,
                  row_attrs, col_attrs)

    with loompy.connect(loom_file_path) as dsout:
        for sample in tqdm(sample_list[1:]):
            filepath = sample[2]
            # +2 because the indexing is 1-based and the first 
            # line is a header
            line = getline(filepath, line_number+2)
            """
            ['ORF_ID',
            'ORF_type',
            'status',
            'phase_score',
            'read_count',
            'length',
            'valid_codons',
            'transcript_id',
            'transcript_type',
            'gene_id',
            'gene_name',
            'gene_type',
            'chrom',
            'strand',
            'start_codon',
            'profile']
            """
            line_splitted = line.split('\t')
            file_orf_id = line_splitted[0]
            assert file_orf_id == annotation_orf_id
            profile = eval(line_splitted[-1])
            col_attrs = { "sample":  np.array([sample[0]])}
            profile = np.array([profile]).reshape(len(row_attrs['chr_pos']), 1)

            dsout.add_columns(profile, col_attrs=col_attrs)# row_attrs=row_attrs)
    #break


 46%|████▌     | 60/131 [04:31<05:18,  4.49s/it]

In [ ]:
transcript_id = 'ENST00000626826.1'
orfs_for_txid = annotation[annotation.transcript_id == transcript_id]

"""
# Stra
#for orf in orfs
loompy.create(filename, matrix, row_attrs, col_attrs)
with loompy.new("outfile.loom") as dsout:
    for sample in samples:
        with loompy.connect(sample) as dsin:
            logging.info(f"Appending {sample}.")
            dsout.add_columns(ds.layers, col_attrs=dsin.col_attrs, row_attrs=dsin.row_attrs)
"""
out_root_dir = '/staging/as/skchoudh/ribopod_hg38_looms/'
for sample_index, sample in enumerate(tqdm(sample_list)):
    df = pd.read_table(sample[2], usecols=['ORF_ID', 'profile'])
    for orf_id
    df.apply()
    
    annotation_orf_id = row['ORF_ID']
    out_dir = os.path.join(out_root_dir, annotation_orf_id)
    mkdir_p(out_dir)
    loom_file_path = os.path.join(out_dir, 'loomfile.loom')
    orf_index = create_index_for_annotation_row(row)
    chrom = row['chrom']
    sample = sample_list[0]      
    filepath = sample[2]
    line = getline(filepath, line_number+2)
    line_splitted = line.split('\t')
    file_orf_id = line_splitted[0]
    assert file_orf_id == annotation_orf_id
    profile = eval(line_splitted[-1])

    row_attrs = ['{}_{}'.format(chrom, pos) for pos in orf_index]

    row_attrs = { "chr_pos": np.array(row_attrs) }
    col_attrs = { "sample":  np.array([sample[0]])}
    profile = np.array([profile]).reshape(len(row_attrs['chr_pos']), 1)
    loompy.create(loom_file_path, profile,
                  row_attrs, col_attrs)

    with loompy.connect(loom_file_path) as dsout:
        for sample in tqdm(sample_list[1:]):
            filepath = sample[2]
            # +2 because the indexing is 1-based and the first 
            # line is a header
            line = getline(filepath, line_number+2)
            """
            ['ORF_ID',
            'ORF_type',
            'status',
            'phase_score',
            'read_count',
            'length',
            'valid_codons',
            'transcript_id',
            'transcript_type',
            'gene_id',
            'gene_name',
            'gene_type',
            'chrom',
            'strand',
            'start_codon',
            'profile']
            """
            line_splitted = line.split('\t')
            file_orf_id = line_splitted[0]
            assert file_orf_id == annotation_orf_id
            profile = eval(line_splitted[-1])
            col_attrs = { "sample":  np.array([sample[0]])}
            profile = np.array([profile]).reshape(len(row_attrs['chr_pos']), 1)

            dsout.add_columns(profile, col_attrs=col_attrs)# row_attrs=row_attrs)
    #break


In [52]:
with loompy.connect('/staging/as/skchoudh/ribopod_hg38_looms/ENST00000626826.1_102197843_102197914_72/loomfile.loom') as ds:
    print(list(ds.col_attrs['SRP']))

['SRP010679', 'SRP010679', 'SRP010679', 'SRP010679', 'SRP010679', 'SRP010679', 'SRP010679', 'SRP010679', 'SRP010679', 'SRP010679', 'SRP010679', 'SRP010679', 'SRP029589', 'SRP029589', 'SRP029589', 'SRP063852', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP098789', 'SRP092068', 'SRP092068', 'SRP092068', 'SRP092068', 'SRP002605', 'SRP002605', 'SRP002605', 'SRP017942', 'SRP017942', 'SRP017942', 'SRP017942', 'SRP031501', 'SRP031501', 'SRP031501', 'SRP031501', 'SRP031501', 'SRP031501', 'SRP038695', 'SRP038695']


In [104]:
def batch(iterable, n=50):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def read_batch_tsv(sample_list):
    dfs = [pd.read_table(tsv,
                         usecols=['ORF_ID', 'profile'], 
                         dtype={'ORF_ID':'str', 'profile': 'str'}, 
                         memory_map=True) for srp, srx, tsv in sample_list]
    srps = [srp for srp, srx, tsv in sample_list]
    srxs = [srx for srp, srx, tsv in sample_list]
    col_attrs = {"study": np.array(srps), "experiment": np.array(srxs) }
    return dfs, col_attrs

def write_loom_file(loom_file_path, matrix, col_attrs, row_attrs=None):
    """Create/Update loom file.
    
    Parameters
    ----------
    filepath: string
              Path to write loomfile
    matrix: array_like
            Data matrix
    col_attrs: dict
               A dict of lists with same length as the columns in matrix
    row_attrs: dict
               A dict of lists with same length as the rows in matrix
    
    """
    # Check if loomfile exists
    if os.path.isfile(loom_file_path):
        # Read its columns attributes
        # to see if the new column_attrs are same as previous
        with loompy.connect(loom_file_path) as ds:
            assert sorted(list(col_attrs.keys())) == sorted(list(ds.col_attrs.keys()))
            # Check if nothing is being added again
            for key in col_attrs.keys():
                assert len(set(col_attrs[key]).intersection(set(ds.col_attrs[key]))) == 0
            
        # Add columns
        with loompy.connect(loom_file_path) as dsout:
            dsout.add_columns(matrix, col_attrs=col_attrs)# row_attrs=row_attrs)
    else:
        loompy.create(loom_file_path,
                      matrix,
                      row_attrs=row_attrs,
                      col_attrs=col_attrs)

def get_row_attrs_from_orf(annotation, orf_id):
    orf_row = annotation.loc[annotation.ORF_ID==orf_id].iloc[0]
    orf_index = create_index_for_annotation_row(orf_row)
    chrom = orf_row['chrom']
    row_attrs = ['{}_{}'.format(chrom, pos) for pos in orf_index]
    row_attrs = { "chr_pos": np.array(row_attrs) }
    return row_attrs


def write_loom_for_dfs(loom_file_path, list_of_dfs, col_attrs, row_attrs, orf_id):
    # Given a list of dfs,
    dfs_subset = [df[df.ORF_ID==orf_id].iloc[0] for df in list_of_dfs]
    profile_stacked = [literal_eval(df.profile) for df in dfs_subset]
    matrix = np.array(profile_stacked).T
    write_loom_file(loom_file_path, matrix, col_attrs, row_attrs)

    
    
def write_loom_batches(sample_list, annotation_filepath, batch_size=50):
    print('Reading annotation ... ')
    annotation = pd.read_table(annotation_filepath)
    print('Done!')
    out_root_dir = '/staging/as/skchoudh/ribopod_hg38_looms/'

    ORF_IDS = annotation.ORF_ID.tolist()
    TX_IDS = annotation.transcript_id.tolist()
    for batch_sample in batch(sample_list, 5):
        # Read all the tsvs in this batch
        print('Reading batch tsv ... ')
        list_of_dfs, col_attrs = read_batch_tsv(batch_sample)        
        print('Done! ')
        # For each ORF in all tsvs
        # write a loom file
        # organized by `transcript_id/ORF_ID.loom`
        for tx_id, orf_id in zip(TX_IDS, ORF_IDS):
            row_attrs = get_row_attrs_from_orf(annotation, orf_id)            
            out_dir = os.path.join(out_root_dir, tx_id)
            mkdir_p(out_dir)
            loom_file_path = os.path.join(out_dir, '{}.loom'.format(orf_id))
            write_loom_for_dfs(loom_file_path, list_of_dfs, col_attrs, row_attrs, orf_id)
        del list_of_dfs
        del col_attrs

In [105]:
write_loom_batches(sample_list, annotation_filepath, batch_size=30)

Reading annotation ... 
Done!
Reading batch tsv ... 
Done! 


AssertionError: 

In [34]:
%%timeit
df = pd.read_table('/staging/as/skchoudh/re-ribo-analysis/hg38/SRP010679/ribocop_results_Feb2019_longest/SRX118286_translating_ORFs.tsv',
                            usecols=['ORF_ID', 'profile'])

13.6 s ± 64.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%%timeit

df = pd.read_table('/staging/as/skchoudh/re-ribo-analysis/hg38/SRP010679/ribocop_results_Feb2019_longest/SRX118286_translating_ORFs.tsv',
                   usecols=['ORF_ID', 'profile'], 
                   dtype={'ORF_ID':'str', 'profile': 'str'}, 
                   memory_map=True)


13.3 s ± 61.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit profiles = df.profile.apply(lambda x: eval(x))

In [110]:
loom_file_path = '/staging/as/skchoudh/ribopod_hg38_looms/ENST00000487959.5/ENST00000487959.5_147844554_147844688_135.loom'
with loompy.connect(loom_file_path) as ds:
    print(ds.values)

AttributeError: 'LoomConnection' object has no attribute 'values'

In [36]:
filepath = '/staging/as/skchoudh/re-ribo-analysis/hg38/SRP010679/ribocop_results_Feb2019_longest/SRX118286_translating_ORFs.tsv'

In [43]:
%timeit
def test_getline(filepath, list_of_lines):
    for line_number in tqdm(list_of_lines):
        line = getline(filepath, line_number+2)
        line_splitted = line.split('\t')
        file_orf_id = line_splitted[0]
        profile = eval(line_splitted[-1])
test_getline(filepath, range(df.shape[0]))

  3%|▎         | 67875/2397475 [00:52<26:25, 1469.25it/s]

KeyboardInterrupt: 

  3%|▎         | 67875/2397475 [01:10<26:25, 1469.25it/s]